## Audio Quality Dataset

In [360]:
import pandas as pd
import numpy as np
import json
from sklearn.metrics import accuracy_score

In [361]:
df = pd.read_csv('student_audio_quality_review.csv')

In [362]:
df.head()

,Standard,Grade,type,answer_text,questionattemptid,total_rubric_score,Audio Link,Transcription,whisper_words,whisper_segments,whisper_language,Audio Audible,Transcript Acceptable
0,ca,1,speaking,https://summitk12-quiz-audios.s3-accelerate.am...,125420729,0,https://drive.google.com/file/d/1l0sWVznEn_f1o...,The kids are cleaning.,"[{""word"": ""The"", ""start"": 0.62, ""end"": 0.8}, {...","[{""id"": 0, ""seek"": 0, ""start"": 0, ""end"": 3.14,...",Icelandic,Yes,Yes
1,ca,2,speaking,https://summitk12-quiz-audios.s3-accelerate.am...,132724395,0,https://drive.google.com/file/d/1gXqLvabNA9C1P...,. .,"[{""word"": ""."", ""start"": 0, ""end"": 1.08}, {""wor...","[{""id"": 0, ""seek"": 0, ""start"": 0, ""end"": 4.4, ...",English,No,No
2,ca,3,speaking,https://summitk12-quiz-audios.s3-accelerate.am...,154839753,0,https://drive.google.com/file/d/16DS157VCuQikP...,The children are in a forest on a log.,"[{""word"": ""The"", ""start"": 1.86, ""end"": 2.04}, ...","[{""id"": 0, ""seek"": 0, ""start"": 0, ""end"": 7.2, ...",Icelandic,Yes,Yes
3,ca,4,speaking,https://summitk12-quiz-audios.s3-accelerate.am...,95970423,0,https://drive.google.com/file/d/10P9XXQpP93WPz...,Það er mamið. Það er kynnt.,"[{""word"": ""\u00dea\u00f0"", ""start"": 0.02, ""end...","[{""id"": 0, ""seek"": 0, ""start"": 0, ""end"": 1.96,...",Icelandic,No,No
4,ca,5,speaking,https://summitk12-quiz-audios.s3-accelerate.am...,16975931,0,https://drive.google.com/file/d/1FVZwuJm5ks7N2...,It's music class.,"[{""word"": ""It's"", ""start"": 0.76, ""end"": 1.3}, ...","[{""id"": 0, ""seek"": 0, ""start"": 0, ""end"": 2.64,...",Icelandic,Yes,Yes


In [363]:
# Rows and Columns

print(f"Rows: {df.shape[0]}")
print(f"Columns: {df.shape[1]} - {list(df.columns)}")

Rows: 1028
Columns: 13 - ['Standard', 'Grade', 'type', 'answer_text', 'questionattemptid', 'total_rubric_score', 'Audio Link', 'Transcription', 'whisper_words', 'whisper_segments', 'whisper_language', 'Audio Audible', 'Transcript Acceptable']


In [364]:
# Checking if any nulls

df.isnull().sum()

Standard                   0
Grade                      0
type                       0
answer_text                0
questionattemptid          0
total_rubric_score         0
Audio Link                 0
Transcription              0
whisper_words              0
whisper_segments           0
whisper_language           0
Audio Audible            196
Transcript Acceptable    196
dtype: int64

In [365]:
# Drop all rows where Audio Audible is Null

df = df.dropna(subset=['Audio Audible'])

In [366]:
# Convert 'Yes'/'No' values to binary (1/0)

df['Audio Audible'] = df['Audio Audible'].map({'Yes':1, 'No':0})
df['Transcript Acceptable'] = df['Transcript Acceptable'].map({'Yes':1, 'No':0})

In [367]:
# 'Ground Truth' is AND of previous two columns

df['ground_truth'] = df['Audio Audible'] & df['Transcript Acceptable']

In [368]:
df.head()

,Standard,Grade,type,answer_text,questionattemptid,total_rubric_score,Audio Link,Transcription,whisper_words,whisper_segments,whisper_language,Audio Audible,Transcript Acceptable,ground_truth
0,ca,1,speaking,https://summitk12-quiz-audios.s3-accelerate.am...,125420729,0,https://drive.google.com/file/d/1l0sWVznEn_f1o...,The kids are cleaning.,"[{""word"": ""The"", ""start"": 0.62, ""end"": 0.8}, {...","[{""id"": 0, ""seek"": 0, ""start"": 0, ""end"": 3.14,...",Icelandic,1,1,1
1,ca,2,speaking,https://summitk12-quiz-audios.s3-accelerate.am...,132724395,0,https://drive.google.com/file/d/1gXqLvabNA9C1P...,. .,"[{""word"": ""."", ""start"": 0, ""end"": 1.08}, {""wor...","[{""id"": 0, ""seek"": 0, ""start"": 0, ""end"": 4.4, ...",English,0,0,0
2,ca,3,speaking,https://summitk12-quiz-audios.s3-accelerate.am...,154839753,0,https://drive.google.com/file/d/16DS157VCuQikP...,The children are in a forest on a log.,"[{""word"": ""The"", ""start"": 1.86, ""end"": 2.04}, ...","[{""id"": 0, ""seek"": 0, ""start"": 0, ""end"": 7.2, ...",Icelandic,1,1,1
3,ca,4,speaking,https://summitk12-quiz-audios.s3-accelerate.am...,95970423,0,https://drive.google.com/file/d/10P9XXQpP93WPz...,Það er mamið. Það er kynnt.,"[{""word"": ""\u00dea\u00f0"", ""start"": 0.02, ""end...","[{""id"": 0, ""seek"": 0, ""start"": 0, ""end"": 1.96,...",Icelandic,0,0,0
4,ca,5,speaking,https://summitk12-quiz-audios.s3-accelerate.am...,16975931,0,https://drive.google.com/file/d/1FVZwuJm5ks7N2...,It's music class.,"[{""word"": ""It's"", ""start"": 0.76, ""end"": 1.3}, ...","[{""id"": 0, ""seek"": 0, ""start"": 0, ""end"": 2.64,...",Icelandic,1,1,1


In [369]:
# Function to extract mean values
def extract_means(segments):
    avg_logprob = [dct["avg_logprob"] for dct in segments if "avg_logprob" in dct]
    compression_ratio = [dct["compression_ratio"] for dct in segments if "compression_ratio" in dct]
    no_speech_prob = [dct["no_speech_prob"] for dct in segments if "no_speech_prob" in dct]

    return (
        np.mean(avg_logprob),
        np.mean(compression_ratio),
        np.mean(no_speech_prob)
    )

In [370]:
# Temp column with parsed JSON data
df["whisper_segments_parsed"] = df["whisper_segments"].apply(
    lambda x: json.loads(x)
)

In [371]:
# Apply 'extract_means' on the parsed column
df[["mean_avg_logprob", "mean_compression_ratio", "mean_no_speech_prob"]] = (
    df["whisper_segments_parsed"]
    .apply(extract_means)
    .apply(pd.Series)
)

In [372]:
# Drop temp column
df.drop(columns=["whisper_segments_parsed"], inplace=True)

In [373]:
df.head()

,Standard,Grade,type,answer_text,questionattemptid,total_rubric_score,Audio Link,Transcription,whisper_words,whisper_segments,whisper_language,Audio Audible,Transcript Acceptable,ground_truth,mean_avg_logprob,mean_compression_ratio,mean_no_speech_prob
0,ca,1,speaking,https://summitk12-quiz-audios.s3-accelerate.am...,125420729,0,https://drive.google.com/file/d/1l0sWVznEn_f1o...,The kids are cleaning.,"[{""word"": ""The"", ""start"": 0.62, ""end"": 0.8}, {...","[{""id"": 0, ""seek"": 0, ""start"": 0, ""end"": 3.14,...",Icelandic,1,1,1,-0.046498,0.733333,1.837492e-10
1,ca,2,speaking,https://summitk12-quiz-audios.s3-accelerate.am...,132724395,0,https://drive.google.com/file/d/1gXqLvabNA9C1P...,. .,"[{""word"": ""."", ""start"": 0, ""end"": 1.08}, {""wor...","[{""id"": 0, ""seek"": 0, ""start"": 0, ""end"": 4.4, ...",English,0,0,0,-2.041714,0.272727,5.108253e-10
2,ca,3,speaking,https://summitk12-quiz-audios.s3-accelerate.am...,154839753,0,https://drive.google.com/file/d/16DS157VCuQikP...,The children are in a forest on a log.,"[{""word"": ""The"", ""start"": 1.86, ""end"": 2.04}, ...","[{""id"": 0, ""seek"": 0, ""start"": 0, ""end"": 7.2, ...",Icelandic,1,1,1,-0.189879,0.904762,2.074286e-10
3,ca,4,speaking,https://summitk12-quiz-audios.s3-accelerate.am...,95970423,0,https://drive.google.com/file/d/10P9XXQpP93WPz...,Það er mamið. Það er kynnt.,"[{""word"": ""\u00dea\u00f0"", ""start"": 0.02, ""end...","[{""id"": 0, ""seek"": 0, ""start"": 0, ""end"": 1.96,...",Icelandic,0,0,0,-0.640734,0.941176,2.977847e-10
4,ca,5,speaking,https://summitk12-quiz-audios.s3-accelerate.am...,16975931,0,https://drive.google.com/file/d/1FVZwuJm5ks7N2...,It's music class.,"[{""word"": ""It's"", ""start"": 0.76, ""end"": 1.3}, ...","[{""id"": 0, ""seek"": 0, ""start"": 0, ""end"": 2.64,...",Icelandic,1,1,1,-0.351254,0.680000,3.434895e-10


In [374]:
# Calculate Min and Max

min_avg_logprob = df['mean_avg_logprob'].min()
min_compress_ratio = df['mean_compression_ratio'].min()
min_no_speech_prob = df['mean_no_speech_prob'].min()

max_avg_logprob = df['mean_avg_logprob'].max()
max_compress_ratio = df['mean_compression_ratio'].max()
max_no_speech_prob = df['mean_no_speech_prob'].max()

In [375]:
# Create Threshold Ranges

log_prob_thresh = np.linspace(min_avg_logprob, max_avg_logprob, 10)
print(log_prob_thresh)

compress_ratio_thresh = np.linspace(min_compress_ratio, max_compress_ratio, 10)
print(compress_ratio_thresh)

log_min = np.log10(min_no_speech_prob)
log_max = np.log10(max_no_speech_prob)

no_speech_prob_thresh = np.logspace(log_min, log_max, 10)
print(no_speech_prob_thresh)


[-8.62453    -7.66983715 -6.7151443  -5.76045145 -4.8057586  -3.85106574
 -2.89637289 -1.94168004 -0.98698719 -0.03229434]
[0.11111111 0.32887295 0.5466348  0.76439664 0.98215848 1.19992033
 1.41768217 1.63544401 1.85320586 2.0709677 ]
[4.41488430e-13 1.64796612e-12 6.15144620e-12 2.29618133e-11
 8.57107179e-11 3.19936717e-10 1.19424391e-09 4.45781445e-09
 1.66399087e-08 6.21126260e-08]


In [376]:
# Find the best threshold combination for maximum accuracy

best_accuracy = 0
best_thresholds = None

for log_prob in log_prob_thresh:
    for compress_ratio in compress_ratio_thresh:
        for nospeech_prob in no_speech_prob_thresh:

            def classify(row):
                if (
                    row['mean_avg_logprob'] > log_prob or
                    row['mean_compression_ratio'] < compress_ratio or
                    row['mean_no_speech_prob'] < nospeech_prob   
                ):
                    return 1
                return 0

            df['temp_output'] = df.apply(classify, axis=1)
    
            acc = accuracy_score(df["ground_truth"], df["temp_output"])
    
            if acc > best_accuracy:
                best_accuracy = acc
                best_thresholds = (log_prob, compress_ratio, nospeech_prob)
                # print(f"New best accuracy: {best_accuracy:.4f} at thresholds {best_thresholds}")

In [377]:
df['temp_output'].value_counts()

temp_output
1    832
Name: count, dtype: int64

In [378]:
df.drop(columns=["temp_output"], inplace=True)

In [379]:
# Apply best thresholds to get final output
def final_classify(row):
    if (
        row["mean_avg_logprob"] > best_thresholds[0] or
        row["mean_compression_ratio"] < best_thresholds[1] or
        row["mean_no_speech_prob"] < best_thresholds[2]
    ):
        return 1
    return 0

df["predicted_output"] = df.apply(final_classify, axis=1)


In [380]:
df["predicted_output"].value_counts()

predicted_output
0    553
1    279
Name: count, dtype: int64